# Single Source Shortest Path (SSSP)

In this notebook, we will use cuGraph to compute the shortest path from a starting vertex to every other vertex in our training dataset.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* available since release 0.6
* Last Edit: 06/22/2022


RAPIDS Versions: 22.08  

Test Hardware

* Tesla V100 32G, CUDA 11.5




## Introduction

Single source shortest path computes the shortest paths from the given starting vertex to all other reachable vertices. 

To compute SSSP for a graph in cuGraph we use:
**cugraph.sssp(G, source)**

Input
* __G__: cugraph.Graph object
* __source__: int, Index of the source vertex

Returns 
* __df__:  a cudf.DataFrame object with two columns:
    * df['vertex']: The vertex identifier for the vertex
    * df['distance']: The computed distance from the source vertex to this vertex
    * df['predecessor']: The predecessor vertex along this paths.  Allows paths to be recreated


### Some notes about vertex IDs...
* The current version of cuGraph requires that vertex IDs be representable as 32-bit integers, meaning graphs currently can contain at most 2^32 unique vertex IDs. However, this limitation is being actively addressed and a version of cuGraph that accommodates more than 2^32 vertices will be available in the near future.
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  
__Note__: The Karate dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  

In [1]:
# Import needed libraries
import cudf
import cugraph

### Create an Edgelist

In [5]:
# Import a built-in dataset
from cugraph.experimental.datasets import karate

gdf = karate.get_edgelist()

In [6]:
# The SSSP algorithm requires that there are weights.  Just use 1.0 here (equivalent to BFS)
gdf["data"] = 1.0

In [7]:
gdf.head()

,src,dst,data
0,1,2,1.0
1,1,3,1.0
2,1,4,1.0
3,1,5,1.0
4,1,6,1.0


### Create a Graph and call SSSP

In [8]:
# create a Graph 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='data')

In [9]:
# Call cugraph.sssp to get the distances from vertex 1:
df = cugraph.sssp(G, 1)

In [10]:
# Print the paths
for index, row in df.to_pandas().iterrows():
    v = int(row['vertex'])
    p = cugraph.utils.get_traversed_path_list(df, v)
    print(v, ': ', p)


5 :  [5, 1]
11 :  [11, 1]
20 :  [20, 1]
25 :  [25, 32, 1]
26 :  [26, 32, 1]
29 :  [29, 3, 1]
10 :  [10, 3, 1]
13 :  [13, 1]
15 :  [15, 34, 32, 1]
16 :  [16, 34, 32, 1]
17 :  [17, 6, 1]
18 :  [18, 1]
19 :  [19, 34, 32, 1]
21 :  [21, 34, 32, 1]
22 :  [22, 1]
23 :  [23, 34, 32, 1]
34 :  [34, 32, 1]
33 :  [33, 3, 1]
3 :  [3, 1]
2 :  [2, 1]
4 :  [4, 1]
32 :  [32, 1]
9 :  [9, 1]
14 :  [14, 1]
24 :  [24, 34, 32, 1]
6 :  [6, 1]
7 :  [7, 1]
8 :  [8, 1]
30 :  [30, 34, 32, 1]
31 :  [31, 2, 1]
28 :  [28, 3, 1]
1 :  [1]
27 :  [27, 34, 32, 1]
12 :  [12, 1]


There are a number of vertices with the same distance of 3

___
Copyright (c) 2019-2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___